In [2]:
!pip install tensorflow

     |████████████████████████████████| 479.6 MB 26 kB/s  eta 0:00:013   |▉                               | 12.6 MB 1.4 MB/s eta 0:05:41     |███████                         | 104.1 MB 1.9 MB/s eta 0:03:17     |████████▍                       | 126.0 MB 5.4 MB/s eta 0:01:06               | 138.7 MB 3.8 MB/s eta 0:01:31     |█████████▉                      | 148.2 MB 40.1 MB/s eta 0:00:09     |███████████▉                    | 177.8 MB 1.5 MB/s eta 0:03:21     |██████████████▏                 | 211.9 MB 1.5 MB/s eta 0:02:58     |███████████████                 | 225.1 MB 1.9 MB/s eta 0:02:12     |███████████████                 | 225.5 MB 1.9 MB/s eta 0:02:11     |███████████████▏                | 226.8 MB 1.9 MB/s eta 0:02:11    |████████████████                | 240.0 MB 4.0 MB/s eta 0:01:01��██████               | 254.0 MB 3.0 MB/s eta 0:01:15  | 254.5 MB 3.0 MB/s eta 0:01:15     |█████████████████               | 255.9 MB 444 kB/s eta 0:08:23     |█████████████████▊              | 2

In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler

# -----------------------------
# PARAMETERS
# -----------------------------
window_size = 50
num_features = 3
num_samples = 1000

# -----------------------------
# 1️⃣ SYNTHETIC SENSOR DATA
# -----------------------------
ax = np.random.normal(0, 0.02, num_samples)
ay = np.random.normal(0, 0.02, num_samples)
az = np.random.normal(9.81, 0.02, num_samples)
current = np.random.normal(0.45, 0.01, num_samples)

# -----------------------------
# 2️⃣ SLIDING WINDOW FEATURES
# -----------------------------
features = []
for i in range(len(ax) - window_size + 1):
    win_ax = ax[i:i+window_size]
    win_ay = ay[i:i+window_size]
    win_az = az[i:i+window_size]
    win_current = current[i:i+window_size]
    
    rms_v = np.sqrt(np.mean(win_ax**2 + win_ay**2 + win_az**2))
    peak_v = np.max(np.sqrt(win_ax**2 + win_ay**2 + win_az**2))
    mean_i = np.mean(win_current)
    
    features.append([rms_v, peak_v, mean_i])

features = np.array(features, dtype=np.float32)

# -----------------------------
# 3️⃣ NORMALIZE FEATURES
# -----------------------------
scaler = MinMaxScaler()
features_scaled = scaler.fit_transform(features)

# -----------------------------
# 4️⃣ AUTOENCODER
# -----------------------------
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(num_features,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(num_features)
])
model.compile(optimizer='adam', loss='mse')

# -----------------------------
# 5️⃣ TRAIN AUTOENCODER
# -----------------------------
history = model.fit(features_scaled, features_scaled, epochs=20, batch_size=16, verbose=0)

# -----------------------------
# 6️⃣ COMPUTE TRAINING RECONSTRUCTION ERROR
# -----------------------------
recon = model.predict(features_scaled)
mse = np.mean((features_scaled - recon)**2, axis=1)

threshold = np.mean(mse) + 3 * np.std(mse)
print(f"Reconstruction error threshold for anomaly detection: {threshold:.6f}")

# -----------------------------
# 7️⃣ OPTIONAL: MODEL 'ACCURACY' ESTIMATE
# -----------------------------
# Simulate 5% anomalies
num_anomalies = int(0.05 * len(features_scaled))
anomalies = features_scaled[:num_anomalies] + np.random.normal(0, 0.2, (num_anomalies, num_features))

# Compute MSE for anomalies
recon_anomalies = model.predict(anomalies)
mse_anomalies = np.mean((anomalies - recon_anomalies)**2, axis=1)

# Detection: True Positive = anomaly detected, True Negative = normal detected
true_positive = np.sum(mse_anomalies > threshold)
true_negative = np.sum(mse[num_anomalies:] <= threshold)
accuracy = (true_positive + true_negative) / len(features_scaled)
print(f"Estimated model detection 'accuracy': {accuracy*100:.2f}%")

# -----------------------------
# 8️⃣ QUANTIZE & SAVE MODEL AS model.h
# -----------------------------
def representative_dataset():
    for i in range(0, len(features_scaled), window_size):
        yield [features_scaled[i:i+window_size].astype(np.float32)]

converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

tflite_quant_model = converter.convert()

c_file = "model.h"
hex_array = ','.join([str(b) for b in tflite_quant_model])
with open(c_file, "w") as f:
    f.write("const unsigned char model_tflite[] = {")
    f.write(hex_array)
    f.write("};\n")
    f.write(f"const unsigned int model_tflite_len = {len(tflite_quant_model)};\n")

print("Quantized TFLite model saved as model.h, ready for ESP32!")


30/30 [==============================] - 0s 2ms/step
Reconstruction error threshold for anomaly detection: 0.000528
2/2 [==============================] - 0s 3ms/step
Estimated model detection 'accuracy': 95.37%
INFO:tensorflow:Assets written to: /tmp/tmp4kzuqexy/assets


INFO:tensorflow:Assets written to: /tmp/tmp4kzuqexy/assets


Quantized TFLite model saved as model.h, ready for ESP32!


/home/mariam/NTI/analysis/myenv/lib/python3.8/site-packages/tensorflow/lite/python/convert.py:887: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
2025-12-29 13:29:14.668445: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2025-12-29 13:29:14.668485: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
2025-12-29 13:29:14.668740: I tensorflow/cc/saved_model/reader.cc:45] Reading SavedModel from: /tmp/tmp4kzuqexy
2025-12-29 13:29:14.669767: I tensorflow/cc/saved_model/reader.cc:91] Reading meta graph with tags { serve }
2025-12-29 13:29:14.669783: I tensorflow/cc/saved_model/reader.cc:132] Reading SavedModel debug info (if present) from: /tmp/tmp4kzuqexy
2025-12-29 13:29:14.674147: I tensorflow/cc/saved_model/loader.cc:231] Restoring SavedModel bundle.
2025-12-29 13:29:14.727005: I tensorflow/cc/saved_model/loader.cc:215]